## This notebook is an example of how to pipeline two models. 
A video stream from a local camera is processed by the face detection model. The face detection results are then processed by the mask detection model, one face bounding box at a time.
Combined result is then displayed.

**Access to camera is required to run this sample.**

This script works with the following inference options:

1. [DeGirum Cloud Platform](https://cs.degirum.com),
1. DeGirum-hosted AI server node shared via Peer-to-Peer VPN,
1. AI server node hosted by you in your local network,
1. AI server running on your local machine,
1. DeGirum ORCA accelerator directly installed on your local machine.

To try different options, you just need to change the `inference_option` in the code below.

The script needs either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning `camera_id` or in .env file by defining `CAMERA_ID` variable and assigning `camera_id = None`.

### Specify where do you want to run your inferences and camera index here

In [ ]:
inference_option = 1  # <<< change it according to your needs selecting from the list in the header comment
camera_id = 0         # camera index or URL; 0 to use default local camera, None to take from .env file

### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools, cv2

In [ ]:
# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")
mask_det_model = zoo.load_model("mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca_1")

# adjust mask model properties
mask_det_model.output_top_k = 1 # report single result with top confidence
mask_det_model.image_backend = 'opencv' 
mask_det_model.input_numpy_colorspace = 'BGR'
mask_det_model._model_parameters.InputImgFmt = ['JPEG']

# adjust face model properties
face_det_model.output_confidence_threshold = 0.7
face_det_model.overlay_font_scale = 1.5
face_det_model.overlay_alpha = 1
face_det_model.image_backend = 'opencv'
face_det_model._model_parameters.InputImgFmt = ['JPEG']

In [ ]:
# AI prediction loop
# Press 'x' or 'q' to stop
with mytools.Display("Faces") as display, \
     mytools.open_video_stream(camera_id) as stream:
    
    with mask_det_model: # performance optimization to keep connection to nested model open
        
        # run face detection model on a camera stream
        for face in face_det_model.predict_batch(mytools.video_source(stream)):
            # prepare list of bboxes of detected faces
            if not face.results:
                continue

            # prepare generator of images cropped around each detected face
            face_crops = ( mytools.Display.crop(face.image, f['bbox']) for f in face.results )

            # for each detected face detect the mask presence
            for masks, face_result in zip(mask_det_model.predict_batch(face_crops), face.results):
                for r in masks.results: # apply mask presence label
                    face_result["label"] = r["label"]

            display.show(face.image_overlay)